In [2]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

import torch, json, re
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration

## Caption Images

In [3]:
MODEL_ID = "llava-hf/llava-1.5-7b-hf"

proc = AutoProcessor.from_pretrained(MODEL_ID)
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    #load_in_4bit=True,          # 4-bit quantization 
    torch_dtype=torch.bfloat16  
).eval()


torch.manual_seed(0)


TAG_INSTR = (
    "You are a tagging engine.\n"
    "Return ONLY a JSON object and nothing else. No prose, no code fences.\n"
    "The JSON must have exactly these fields:\n"
    '{"caption": str, "objects":[str], "attributes":[str], "styles":[str], '
    '"colors":[str]\n'
    "Rules: lowercase; <=10 items TOTAL across all lists; strings only; no nulls."
)

def _decode_json(s: str):
    m = re.search(r"\{.*\}", s, flags=re.S)
    if not m:
        return {"caption":"", **{k:[] for k in ["objects","attributes","styles","colors"]}}
    return json.loads(m.group(0))

def tag_image_llava(pil_img: Image.Image):
    # LLaVA chat template: we use a single turn with an image placeholder
    messages = [
        {"role":"user","content":[{"type":"image","image":pil_img},
                                  {"type":"text","text":TAG_INSTR}]}
    ]
    prompt = proc.apply_chat_template(messages, add_generation_prompt=True)
    inputs = proc(text=[prompt], images=[pil_img], return_tensors="pt").to(model.device)

    with torch.inference_mode():
        ids = model.generate(
            **inputs,
            max_new_tokens=192,
            do_sample=False,   # greedy = reproducible
            top_p=1.0,
        )
    out = proc.batch_decode(ids, skip_special_tokens=True)[0]
    parts = re.split(r'ASSISTANT\s*:\s*', out, flags=re.I)
    return _decode_json(parts[-1])




Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
images = pd.read_csv("../data/flickr/processed/filtered_ratings_df.csv")["imagePair"].unique()
image_dir = "../data/flickr/raw/FLICKR-AES-001/40K/"
image_captions = {"image": [], "caption": [], "objects": [], "attributes": [], "styles": [], "colors": []}
failed_images = []

for image in images:
    try:
        img = Image.open(f"{image_dir}/{image}").convert("RGB")
        desc = tag_image_llava(img)

        row = {
            "image": image,
            "caption": desc.get("caption",""),
            "objects": ", ".join(desc.get("objects", [])),
            "attributes": ", ".join(desc.get("attributes", [])),
            "styles": ", ".join(desc.get("styles", [])),
            "colors": ", ".join(desc.get("colors", [])),
        }

        for k, v in row.items():  
            image_captions[k].append(v)

    except Exception as e:
        print(f"Failed at {image}: {e}")
        for k in image_captions:
            image_captions[k].append("" if k!="image" else image)
        failed_images.append(image)


df = pd.DataFrame(image_captions)
df.to_csv("../data/flickr/processed/captions/image_captions_llava-1.5-7b-hf.csv", index=False)


Failed at farm5_4015_4646430512_36d6fa4e3d.jpg: Expecting ',' delimiter: line 3 column 142 (char 357)
Failed at farm4_3282_2671295429_ba3aebd49d.jpg: Extra data: line 1 column 187 (char 186)
Failed at farm2_1433_5183274968_96aec38e6e.jpg: Unterminated string starting at: line 1 column 13 (char 12)
Failed at farm9_8350_8251215038_15a283f7db_b.jpg: Extra data: line 2 column 1 (char 103)


OSError: Cannot save file into a non-existent directory: 'captions'

It seems like these images are anomalous, and they fail tagging using the VLM. We will drop them:

In [12]:
image_captions_from_failed = {}
for img_name in failed_images:
    try:
        img = Image.open(f"{image_dir}/{img_name}").convert("RGB")
        desc = tag_image_llava(img)
        print("Description", desc)
        row = {
            "image": image,
            "caption": desc.get("caption",""),
            "objects": ", ".join(desc.get("objects", [])),
            "attributes": ", ".join(desc.get("attributes", [])),
            "styles": ", ".join(desc.get("styles", [])),
            "colors": ", ".join(desc.get("colors", [])),
        }
        
        for k, v in row.items():  
            image_captions_from_failed[k].append(v)
    except Exception as e:
        print(f"Reprocess failed at {img_name}: {e}")

Reprocess failed at farm5_4015_4646430512_36d6fa4e3d.jpg: Expecting ',' delimiter: line 3 column 142 (char 357)
Reprocess failed at farm4_3282_2671295429_ba3aebd49d.jpg: Extra data: line 1 column 187 (char 186)
Reprocess failed at farm2_1433_5183274968_96aec38e6e.jpg: Unterminated string starting at: line 1 column 13 (char 12)
Reprocess failed at farm9_8350_8251215038_15a283f7db_b.jpg: Extra data: line 2 column 1 (char 103)


In [14]:
import json, re, pandas as pd
from collections import defaultdict

def split_list_cell(s):
    if not isinstance(s, str) or not s.strip():
        return []
    return [t.strip().lower() for t in s.split(",") if t.strip()]

def shrink_row(row):
    # keep it compact and deterministic
    return {
        "caption": (row.get("caption") or "").strip(),
        "objects": split_list_cell(row.get("objects", "")),
        "attributes": split_list_cell(row.get("attributes", "")),
        "styles": split_list_cell(row.get("styles", "")),
        "colors": split_list_cell(row.get("colors", "")),
    }

def pack_examples(caption_df, image_ids):
    # returns list of compact dicts for the prompt
    sub = caption_df.loc[caption_df["image"].isin(image_ids)]
    return [shrink_row(rec) for rec in sub.to_dict("records")]


In [4]:
ratings = pd.read_csv("../data/flickr/processed/train/train_usrthrs_0.csv")
caps    = pd.read_csv("../data/flickr/processed/captions/image_captions_llava-1.5-7b-hf.csv")

# Fast lookup by image id
caps = caps.fillna("")

In [5]:
caps.head()

,image,caption,objects,attributes,styles,colors
0,farm1_176_396297967_94dae8ad97.jpg,Asian man wearing glasses and a blue shirt,"man, glasses, shirt, blue","wearing, glasses, shirt, blue","casual, formal","blue, white"
1,farm1_106_268532564_c273eb23e6.jpg,Asian men playing drums,"drums, drums, drums, drums, drums, drums, drum...","wood, wood, wood, wood, wood, wood, wood, wood...","traditional, traditional, traditional, traditi...","brown, brown, brown, brown, brown, brown, brow..."
2,farm8_7224_7246491674_540646a594.jpg,Peacock and Ostrich,"peacock, ostrich","blue, white, brown",natural,"blue, white, brown"
3,farm3_2643_4041742092_25f170c501.jpg,Bar scene with people and drinks,"bar, people, drinks, lights, truck, sign, soda...","bar, lights, truck, sign, soda, beer, bottle","bar, lights, truck, sign, soda, beer, bottle","red, green, white, blue, yellow, orange, purpl..."
4,farm4_3080_2922825219_1ba9c41b1a.jpg,A castle with a pointed roof,castle,"roof, pointed",medieval,brown


In [6]:
ratings.tail()

,original_index,Unnamed: 0,worker,imagePair,score,old_worker_id,image_id,worker_id
189003,193202,203098,A75N0N2PWNC9Q,farm5_4095_4927061545_dff4fea34f.jpg,4,209,38021,209
189004,193203,203099,A75N0N2PWNC9Q,farm8_7382_13133462963_5efb9fe9d4_b.jpg,5,209,38022,209
189005,193205,203101,A75N0N2PWNC9Q,farm9_8493_8276626373_d72e32ca2d.jpg,4,209,38024,209
189006,193206,203102,A75N0N2PWNC9Q,farm7_6158_6171447812_6426657080.jpg,4,209,38025,209
189007,193207,203103,A75N0N2PWNC9Q,farm3_2216_2536463126_faa3e86602.jpg,4,209,38026,209


In [14]:

import re, math, json, pandas as pd
from collections import Counter
from pathlib import Path

# ----------------------- CONFIG -----------------------
RATINGS_CSV = "../data/flickr/processed/train/train_usrthrs_0.csv"   # has columns: worker_id, imagePair, score
CAPS_CSV    = "../data/flickr/processed/captions/image_captions_llava-1.5-7b-hf.csv"      # has columns: image, caption, objects, attributes, styles, colors
OUT_JSONL   = "./personas_deterministic.jsonl"
OUT_CSV     = "./personas_deterministic.csv"

USER_COL  = "worker_id"
IMG_COL   = "imagePair"
SCORE_COL = "score"
LIKE_MIN  = 4              # >=4 is like
DISLIKE_MAX = 2            # <=2 is dislike
MAX_PER_USER = 20          # cap per side
TOPK = 12                  # keywords per side
MIN_COUNT = 2              # min frequency on its own side to be considered
SMOOTH_ALPHA = 1.0         # add-alpha smoothing in log-odds
# ------------------------------------------------------

# -------------------- utilities -----------------------
def split_list_cell(s):
    if not isinstance(s, str) or not s.strip():
        return []
    # normalize and dedup while keeping order
    seen = set(); out = []
    for t in (x.strip().lower() for x in s.split(",") if x.strip()):
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def row_to_desc(row):
    return {
        "caption": (row.get("caption") or "").strip(),
        "objects": split_list_cell(row.get("objects", "")),
        "attributes": split_list_cell(row.get("attributes", "")),
        "styles": split_list_cell(row.get("styles", "")),
        "colors": split_list_cell(row.get("colors", "")),
    }

def is_like(score):
    try: return float(score) >= LIKE_MIN
    except: return False

def is_dislike(score):
    try: return float(score) <= DISLIKE_MAX
    except: return False

# Token cleaning: keep only reasonable tokens (1–2 words, alnum+space)
TOKEN_RE = re.compile(r"^[a-z0-9]+(?: [a-z0-9]+)?$")
STOP = {
    "objects","attributes","styles","colors","positive","negative","keyword","keywords",
    "caption","image","photo","photograph","picture","tagging","engine","the","a","an",
    "men","man","women","woman","people","person","persons","kid","child","children",
    "asian","black","white","brown","gold","silver"  # optional: add/remove to taste
}

def clean_token(t):
    if not isinstance(t, str): return None
    s = t.strip().lower()
    s = s.replace("/", " ").replace("_"," ").replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    if s in STOP: return None
    # filter obvious junk like “the, hogan, bar” that slipped as single tokens
    if not TOKEN_RE.match(s): return None
    if len(s) < 2 or len(s) > 20: return None
    return s

def tokens_from_desc(d):
    toks = []
    for k in ("objects","attributes","styles","colors"):
        for t in d.get(k, []) or []:
            c = clean_token(t)
            if c: toks.append(c)
    return toks

def contrastive_keywords(like_descs, dislike_descs, topk=12, min_count=2, alpha=1.0):
    cl = Counter(); cd = Counter()
    for d in like_descs:     cl.update(tokens_from_desc(d))
    for d in dislike_descs:  cd.update(tokens_from_desc(d))

    total_l = sum(cl.values()); total_d = sum(cd.values())
    V = len(set(cl)|set(cd)) or 1

    def pos_score(t):
        return math.log((cl[t] + alpha) / (total_l + alpha*V)) - math.log((cd[t] + alpha) / (total_d + alpha*V))
    def neg_score(t):
        return math.log((cd[t] + alpha) / (total_d + alpha*V)) - math.log((cl[t] + alpha) / (total_l + alpha*V))

    pos_cands = [t for t in cl if cl[t] >= min_count]
    neg_cands = [t for t in cd if cd[t] >= min_count]

    pos_sorted = sorted(pos_cands, key=pos_score, reverse=True)
    neg_sorted = sorted(neg_cands, key=neg_score, reverse=True)

    pos = []
    for t in pos_sorted:
        if t not in neg_sorted:
            pos.append(t)
        if len(pos) == topk: break

    neg = []
    for t in neg_sorted:
        if t not in pos:
            neg.append(t)
        if len(neg) == topk: break

    return pos, neg

def write_persona_text(pos, neg):
    def fmt(lst, k=6): return ", ".join(lst[:k]) if lst else "varied subjects"
    lines = []
    lines.append(f"This user gravitates toward {fmt(pos)}.")
    lines.append(f"They tend to avoid {fmt(neg)}." if neg else "They show few consistent aversions.")
    lines.append("Overall, their taste is inferred directly from prior likes and dislikes.")
    return " ".join(lines)

# ------------------ core pipeline ---------------------
def build_image_desc_lookup(caps_df):
    caps_df = caps_df.fillna("")
    return { rec["image"]: row_to_desc(rec) for rec in caps_df.to_dict("records") }

def get_user_images(ratings_df, uid, max_like=20, max_dislike=20):
    sub = ratings_df.loc[ratings_df[USER_COL] == uid, [IMG_COL, SCORE_COL]]
    likes = []
    dislikes = []
    for img, sc in sub[[IMG_COL, SCORE_COL]].itertuples(index=False):
        if is_like(sc): likes.append(img)
        elif is_dislike(sc): dislikes.append(img)
    return likes[:max_like], dislikes[:max_dislike]

def map_to_descs(image_ids, image2desc):
    descs = []
    for img in image_ids:
        d = image2desc.get(img)
        if d is None: d = {"caption":"", "objects":[], "attributes":[], "styles":[], "colors":[]}
        descs.append(d)
    return descs

def summarize_user(uid, ratings_df, caps_df, image2desc,
                   max_like=MAX_PER_USER, max_dislike=MAX_PER_USER,
                   topk=TOPK, min_count=MIN_COUNT, alpha=SMOOTH_ALPHA):
    likes, dislikes = get_user_images(ratings_df, uid, max_like, max_dislike)
    like_descs     = map_to_descs(likes, image2desc)
    dislike_descs  = map_to_descs(dislikes, image2desc)

    # if absolutely no signal, return empty/neutral
    if not like_descs and not dislike_descs:
        return {
            "user_id": int(uid),
            "likes_used": 0,
            "dislikes_used": 0,
            "persona": "Insufficient data to infer a consistent visual profile.",
            "keywords_positive": [],
            "keywords_negative": [],
            "like_images": likes,
            "dislike_images": dislikes,
        }

    kpos, kneg = contrastive_keywords(like_descs, dislike_descs, topk=topk, min_count=min_count, alpha=alpha)
    persona = write_persona_text(kpos, kneg)

    return {
        "user_id": int(uid),
        "likes_used": len(like_descs),
        "dislikes_used": len(dislike_descs),
        "persona": persona,
        "keywords_positive": kpos,
        "keywords_negative": kneg,
        "like_images": likes,
        "dislike_images": dislikes,
    }

def run_make_personas(ratings_csv=RATINGS_CSV, caps_csv=CAPS_CSV,
                      out_jsonl=OUT_JSONL, out_csv=OUT_CSV):
    ratings = pd.read_csv(ratings_csv)
    caps    = pd.read_csv(caps_csv)

    # sanity columns
    missing_cols = []
    for c in [USER_COL, IMG_COL, SCORE_COL]:
        if c not in ratings.columns: missing_cols.append(("ratings", c))
    for c in ["image","caption","objects","attributes","styles","colors"]:
        if c not in caps.columns: missing_cols.append(("caps", c))
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    image2desc = build_image_desc_lookup(caps)
    user_ids = sorted(ratings[USER_COL].unique().tolist())

    results = []
    failures = []
    for uid in user_ids:
        try:
            res = summarize_user(uid, ratings, caps, image2desc)
            results.append(res)
        except Exception as e:
            failures.append((uid, str(e)))

    # save JSONL
    with open(out_jsonl, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    # save flat CSV
    flat = pd.DataFrame([{
        "user_id": r["user_id"],
        "likes_used": r["likes_used"],
        "dislikes_used": r["dislikes_used"],
        "persona": r["persona"],
        "keywords_positive": ", ".join(r["keywords_positive"]),
        "keywords_negative": ", ".join(r["keywords_negative"]),
    } for r in results])
    flat.to_csv(out_csv, index=False)

    print(f"Wrote {len(results)} users to:")
    print(f"  JSONL: {out_jsonl}")
    print(f"  CSV:   {out_csv}")
    if failures:
        print(f"{len(failures)} users failed (skipped). Examples:")
        for uid, msg in failures[:5]:
            print(f"  user {uid}: {msg}")


In [15]:
run_make_personas()

Wrote 210 users to:
  JSONL: ./personas_deterministic.jsonl
  CSV:   ./personas_deterministic.csv


In [ ]:
# llava_personas.py
# Robust LLaVA-based user profiles with hard fallbacks.
# - Extracts keywords with LLaVA (JSON, hardened parsing + repair)
# - Builds persona with LLaVA (allowed vocab + hardened parsing)
# - If any step fails, falls back to deterministic keywords + template persona
#
# Requirements:
#   transformers, pandas, tqdm
#   A loaded LLaVA processor+model: set MODEL_ID to your LLaVA checkpoint

import os, re, json, math, pandas as pd
from collections import Counter
from pathlib import Path
from tqdm import tqdm
from typing import List, Dict, Tuple

# ----------------------- CONFIG -----------------------
RATINGS_CSV = "../data/flickr/processed/train/train_usrthrs_0.csv"   # has columns: worker_id, imagePair, score
CAPS_CSV    = "../data/flickr/processed/captions/image_captions_llava-1.5-7b-hf.csv"      # has columns: image, caption, objects, attributes, styles, colors
OUT_JSONL   = "./personas_llava.jsonl"
OUT_CSV     = "./personas_llava.csv"

MODEL_ID    = "llava-hf/llava-1.5-7b-hf"   # your model id here
DEVICE_MAP  = "auto"
LOAD_4BIT   = True

# User selection & prompt bounds
LIKE_MIN      = 4
DISLIKE_MAX   = 2
MAX_LIKES     = 10     # keep prompts small & focused
MAX_DISLIKES  = 10
MAX_PER_FIELD = 2      # tokens per field per image in prompt
KW_TOPK       = 12
KW_MIN_COUNT  = 2
SMOOTH_ALPHA  = 1.0

# LLaVA generation
KW_MAX_NEW    = 512
PERS_MAX_NEW  = 384

# ------------------------------------------------------

# -------------------- LLaVA load ----------------------
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

proc = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    #load_in_4bit=True,          # 4-bit quantization 
    torch_dtype=torch.bfloat16  
).eval()


def llava_chat_text_only(text: str, max_new_tokens=512) -> str:
    messages = [{"role":"user","content":[{"type":"text","text":text}]}]
    prompt = proc.apply_chat_template(messages, add_generation_prompt=True)
    inputs = proc(text=[prompt], return_tensors="pt").to(model.device)
    with torch.inference_mode():
        ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return proc.batch_decode(ids, skip_special_tokens=True)[0]

# -------------------- data utils ----------------------
def split_list_cell(s):
    if not isinstance(s, str) or not s.strip(): return []
    seen = set(); out = []
    for t in (x.strip().lower() for x in s.split(",") if x.strip()):
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def row_to_desc(row):
    return {
        "caption": (row.get("caption") or "").strip(),
        "objects": split_list_cell(row.get("objects","")),
        "attributes": split_list_cell(row.get("attributes","")),
        "styles": split_list_cell(row.get("styles","")),
        "colors": split_list_cell(row.get("colors","")),
    }

def build_image_desc_lookup(caps_df):
    caps_df = caps_df.fillna("")
    return { rec["image"]: row_to_desc(rec) for rec in caps_df.to_dict("records") }

def is_like(score):
    try: return float(score) >= LIKE_MIN
    except: return False

def is_dislike(score):
    try: return float(score) <= DISLIKE_MAX
    except: return False

def get_user_images(ratings_df, uid, max_like=MAX_LIKES, max_dislike=MAX_DISLIKES):
    sub = ratings_df.loc[ratings_df["worker_id"]==uid, ["imagePair","score"]]
    likes, dislikes = [], []
    for img, sc in sub[["imagePair","score"]].itertuples(index=False):
        if is_like(sc): likes.append(img)
        elif is_dislike(sc): dislikes.append(img)
    return likes[:max_like], dislikes[:max_dislike]

# -------------------- prompt packing ------------------
def shrink_desc(d, max_per_field=MAX_PER_FIELD):
    return {
        "caption": d.get("caption","")[:100],
        "objects": d.get("objects", [])[:max_per_field],
        "attributes": d.get("attributes", [])[:max_per_field],
        "styles": d.get("styles", [])[:max_per_field],
        "colors": d.get("colors", [])[:max_per_field],
    }

def pack_examples(image_ids: List[str], image2desc: Dict[str,Dict], max_items=10):
    out = []
    for img in image_ids[:max_items]:
        d = image2desc.get(img, {"caption":"", "objects":[], "attributes":[], "styles":[], "colors":[]})
        out.append(shrink_desc(d))
    return out

KW_INSTR = (
  "Infer explicit visual preference keywords from the examples of LIKES and DISLIKES.\n"
  "Return ONLY a JSON object. No prose, no prefixes, no code fences.\n"
  "The JSON MUST have exactly two top-level keys: positive_keywords and negative_keywords.\n"
  "Each MUST be an object with EXACTLY these keys: objects, attributes, styles, colors.\n"
  "Each value MUST be an array of lowercase strings (5–8 ideally; if insufficient evidence, use fewer).\n"
  "Do NOT invent any other keys. Do NOT use placeholders like '...'.\n"
  "VALID OUTPUT EXAMPLE:\n"
  '{"positive_keywords":{"objects":["cat","flower"],"attributes":["vintage"],"styles":["macro"],"colors":["pastel"]},'
  '"negative_keywords":{"objects":["car"],"attributes":["crowded"],"styles":["sporty"],"colors":["neon"]}}'
)

def make_kw_prompt(like_descs, dislike_descs):
    return (
        KW_INSTR
        + "\nLIKES:\n"    + json.dumps(like_descs, ensure_ascii=False)
        + "\nDISLIKES:\n" + json.dumps(dislike_descs, ensure_ascii=False)
    )

def build_allowed_vocab(kw):
    allowed = set()
    for side in ("positive_keywords", "negative_keywords"):
        d = kw.get(side, {}) or {}
        for k in ("objects","attributes","styles","colors"):
            for t in d.get(k, []) or []:
                if isinstance(t, str) and t.strip():
                    allowed.add(t.strip().lower())
    return sorted(allowed)

CATEGORY_WORDS = {"objects","attributes","styles","colors","positive","negative","keyword","keywords"}

def make_persona_prompt(kw):
    allowed = build_allowed_vocab(kw)
    PERSONA_INSTR = (
      "Return ONLY a JSON object (no prose/prefixes).\n"
      "Keys: persona (string), keywords_positive (array of strings), keywords_negative (array of strings).\n"
      "Rules:\n"
      "- Use ONLY tokens from the allowed list; do not invent new words.\n"
      "- Do NOT include meta words like 'objects','attributes','styles','colors'.\n"
      "- keywords_* should contain as many unique lowercase tokens as available (up to ~15 each).\n"
      "- persona: 3–5 short neutral sentences summarizing likes/avoids.\n"
      'VALID OUTPUT EXAMPLE: {"persona":"...", "keywords_positive":["pastel","vintage"], "keywords_negative":["neon","crowded"]}'
    )
    payload = PERSONA_INSTR + "\nAllowed tokens:\n" + json.dumps(allowed, ensure_ascii=False) + "\nInput keywords:\n" + json.dumps(kw, ensure_ascii=False)
    return payload

# -------------------- JSON hardening ------------------
def _after_assistant(text: str) -> str:
    parts = re.split(r'ASSISTANT\s*:\s*', text, flags=re.I)
    return parts[-1] if parts else text

def _largest_json(text: str) -> str | None:
    stack, starts, spans = [], [], []
    for i,ch in enumerate(text):
        if ch == "{": stack.append("{"); starts.append(i)
        elif ch == "}":
            if stack:
                stack.pop(); s = starts.pop()
                if not stack: spans.append((s, i+1))
    if not spans: return None
    s,e = max(spans, key=lambda se: se[1]-se[0])
    return text[s:e]

def _sanitize_json(s: str) -> str:
    t = s.strip().strip("` \n")
    t = t.replace("\\_", "_")
    t = re.sub(r'\\(?!["\\/bfnrtu])', '', t)     # illegal escapes
    t = re.sub(r",\s*([}\]])", r"\1", t)         # trailing commas
    return t

def parse_json_hard(text: str) -> dict:
    reply = _after_assistant(text)
    block = _largest_json(reply)
    if not block:
        raise ValueError("No JSON object found")
    block = _sanitize_json(block)
    return json.loads(block)

# JSON repair roundtrip
def try_fix_with_llava(bad_text: str, schema_hint: str, max_new=256) -> dict:
    fix_instr = (
      "Fix the following into a VALID JSON object that matches this schema exactly.\n"
      "No extra keys, no prose. Return ONLY the JSON.\nSchema:\n" + schema_hint
    )
    messages = [{"role":"user","content":[
        {"type":"text","text":fix_instr},
        {"type":"text","text":bad_text}
    ]}]
    prompt = proc.apply_chat_template(messages, add_generation_prompt=True)
    inputs = proc(text=[prompt], return_tensors="pt").to(model.device)
    with torch.inference_mode():
        ids = model.generate(**inputs, max_new_tokens=max_new, do_sample=False)
    fixed = proc.batch_decode(ids, skip_special_tokens=True)[0]
    return parse_json_hard(fixed)

# Coercers
KW_SCHEMA_EMPTY = {
  "positive_keywords": {"objects":[], "attributes":[], "styles":[], "colors":[]},
  "negative_keywords": {"objects":[], "attributes":[], "styles":[], "colors":[]},
}

def coerce_kw(obj: dict) -> dict:
    # accept \_ variants
    if "negative\\_keywords" in obj and "negative_keywords" not in obj:
        obj["negative_keywords"] = obj.pop("negative\\_keywords")
    out = {"positive_keywords":{}, "negative_keywords":{}}
    for side in ("positive_keywords","negative_keywords"):
        src = obj.get(side, {}) if isinstance(obj.get(side, {}), dict) else {}
        dst = {}
        for k in ("objects","attributes","styles","colors"):
            v = src.get(k, [])
            if not isinstance(v, list): v = []
            dst[k] = [str(x).lower().strip() for x in v if isinstance(x,(str,int,float)) and str(x).strip()]
            dst[k] = dst[k][:8]
        out[side] = {**KW_SCHEMA_EMPTY[side], **dst}
    return out

def coerce_persona(obj: dict, allowed_vocab: set) -> dict:
    # normalize keys
    if "keywords\\_positive" in obj and "keywords_positive" not in obj:
        obj["keywords_positive"] = obj.pop("keywords\\_positive")
    if "keywords\\_negative" in obj and "keywords_negative" not in obj:
        obj["keywords_negative"] = obj.pop("keywords\\_negative")
    persona = obj.get("persona","") if isinstance(obj.get("persona",""), str) else ""
    def clean(lst):
        seen, out = set(), []
        for x in (lst or []):
            if not isinstance(x,(str,int,float)): continue
            t = str(x).lower().strip()
            if (not t) or (t in CATEGORY_WORDS) or (t not in allowed_vocab): continue
            if t in seen: continue
            seen.add(t); out.append(t)
        return out[:15]
    return {
        "persona": persona,
        "keywords_positive": clean(obj.get("keywords_positive", [])),
        "keywords_negative": clean(obj.get("keywords_negative", [])),
    }

# ---------------- deterministic fallback ----------------
TOKEN_RE = re.compile(r"^[a-z0-9]+(?: [a-z0-9]+)?$")
STOP = {
    "objects","attributes","styles","colors","positive","negative","keyword","keywords",
    "caption","image","photo","photograph","picture","tagging","engine","the","a","an",
}

def clean_token(t):
    if not isinstance(t, str): return None
    s = t.strip().lower().replace("/", " ").replace("_"," ").replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    if s in STOP: return None
    if not TOKEN_RE.match(s): return None
    if len(s) < 2 or len(s) > 20: return None
    return s

def tokens_from_desc(d):
    toks = []
    for k in ("objects","attributes","styles","colors"):
        for t in d.get(k, []) or []:
            c = clean_token(t)
            if c: toks.append(c)
    return toks

def deterministic_keywords(like_descs, dislike_descs, topk=KW_TOPK, min_count=KW_MIN_COUNT, alpha=SMOOTH_ALPHA):
    cl = Counter(); cd = Counter()
    for d in like_descs: cl.update(tokens_from_desc(d))
    for d in dislike_descs: cd.update(tokens_from_desc(d))
    total_l = sum(cl.values()); total_d = sum(cd.values()); V = len(set(cl)|set(cd)) or 1
    def pos_score(t):
        return math.log((cl[t]+alpha)/(total_l+alpha*V)) - math.log((cd[t]+alpha)/(total_d+alpha*V))
    def neg_score(t):
        return math.log((cd[t]+alpha)/(total_d+alpha*V)) - math.log((cl[t]+alpha)/(total_l+alpha*V))
    pos_c = [t for t in cl if cl[t] >= min_count]
    neg_c = [t for t in cd if cd[t] >= min_count]
    pos = []; 
    for t in sorted(pos_c, key=pos_score, reverse=True):
        if t not in neg_c: pos.append(t)
        if len(pos)==topk: break
    neg = [];
    for t in sorted(neg_c, key=neg_score, reverse=True):
        if t not in pos: neg.append(t)
        if len(neg)==topk: break
    return pos, neg

def persona_template(pos, neg):
    def fmt(lst, k=6): return ", ".join(lst[:k]) if lst else "varied subjects"
    lines = []
    lines.append(f"This user gravitates toward {fmt(pos)}.")
    lines.append(f"They tend to avoid {fmt(neg)}." if neg else "They show few consistent aversions.")
    lines.append("Overall, their preferences are inferred from prior likes and dislikes.")
    return " ".join(lines)

# ---------------- per-user pipeline -------------------
def run_user(uid, ratings, image2desc):
    likes, dislikes = get_user_images(ratings, uid)
    like_descs  = pack_examples(likes, image2desc, max_items=MAX_LIKES)
    dislike_descs = pack_examples(dislikes, image2desc, max_items=MAX_DISLIKES)

    # If no signal, return neutral
    if not like_descs and not dislike_descs:
        return {
            "user_id": int(uid), "likes_used": 0, "dislikes_used": 0,
            "persona": "Insufficient data to infer a consistent visual profile.",
            "keywords_positive": [], "keywords_negative": [],
            "like_images": likes, "dislike_images": dislikes
        }

    # --- Keywords with LLaVA (robust) ---
    kw_prompt = make_kw_prompt(like_descs, dislike_descs)
    try:
        kw_raw = llava_chat_text_only(kw_prompt, max_new_tokens=KW_MAX_NEW)
        try:
            kw_obj = parse_json_hard(kw_raw)
        except Exception:
            schema = json.dumps(KW_SCHEMA_EMPTY, ensure_ascii=False)
            kw_obj = try_fix_with_llava(kw_raw, schema_hint=schema, max_new=256)
        kw = coerce_kw(kw_obj)
    except Exception as e:
        # fallback to deterministic
        pos, neg = deterministic_keywords(like_descs, dislike_descs)
        persona = persona_template(pos, neg)
        return {
            "user_id": int(uid), "likes_used": len(like_descs), "dislikes_used": len(dislike_descs),
            "persona": persona, "keywords_positive": pos, "keywords_negative": neg,
            "like_images": likes, "dislike_images": dislikes,
            "note": f"keywords fell back: {type(e).__name__}"
        }

    # --- Persona with LLaVA (allowed vocab, robust) ---
    persona_prompt = make_persona_prompt(kw)
    try:
        pers_raw = llava_chat_text_only(persona_prompt, max_new_tokens=PERS_MAX_NEW)
        try:
            pers_obj = parse_json_hard(pers_raw)
        except Exception:
            schema = '{"persona":"","keywords_positive":[],"keywords_negative":[]}'
            pers_obj = try_fix_with_llava(pers_raw, schema_hint=schema, max_new=192)
        allowed = set(build_allowed_vocab(kw)) - CATEGORY_WORDS
        pers = coerce_persona(pers_obj, allowed)
        # if both lists ended empty, soft fallback to deterministic combos
        if not pers["keywords_positive"] and not pers["keywords_negative"]:
            pos, neg = deterministic_keywords(like_descs, dislike_descs)
            if pos or neg:
                pers["keywords_positive"], pers["keywords_negative"] = pos, neg
                if not pers["persona"]:
                    pers["persona"] = persona_template(pos, neg)
    except Exception as e:
        # fallback persona from deterministic
        pos, neg = deterministic_keywords(like_descs, dislike_descs)
        persona = persona_template(pos, neg)
        return {
            "user_id": int(uid), "likes_used": len(like_descs), "dislikes_used": len(dislike_descs),
            "persona": persona, "keywords_positive": pos, "keywords_negative": neg,
            "like_images": likes, "dislike_images": dislikes,
            "note": f"persona fell back: {type(e).__name__}"
        }

    return {
        "user_id": int(uid),
        "likes_used": len(like_descs),
        "dislikes_used": len(dislike_descs),
        "persona": pers.get("persona",""),
        "keywords_positive": pers.get("keywords_positive",[]),
        "keywords_negative": pers.get("keywords_negative",[]),
        "like_images": likes,
        "dislike_images": dislikes,
    }

# ---------------- main runner -------------------------
def main():
    ratings = pd.read_csv(RATINGS_CSV)
    caps    = pd.read_csv(CAPS_CSV)
    # sanity columns
    for c in ["worker_id","imagePair","score"]:
        assert c in ratings.columns, f"ratings missing {c}"
    for c in ["image","caption","objects","attributes","styles","colors"]:
        assert c in caps.columns, f"caps missing {c}"

    image2desc = build_image_desc_lookup(caps)
    user_ids = sorted(ratings["worker_id"].unique().tolist())

    results = []
    for uid in tqdm(user_ids, desc="Obtaining user profiles"):
        try:
            out = run_user(uid, ratings, image2desc)
            results.append(out)
        except Exception as e:
            # even if something slips through, never crash the batch:
            pos, neg = [], []
            try:
                likes, dislikes = get_user_images(ratings, uid)
                like_descs  = pack_examples(likes, image2desc, max_items=MAX_LIKES)
                dislike_descs = pack_examples(dislikes, image2desc, max_items=MAX_DISLIKES)
                pos, neg = deterministic_keywords(like_descs, dislike_descs)
                persona = persona_template(pos, neg)
            except Exception:
                persona = "Profile unavailable due to data error."
            results.append({
                "user_id": int(uid), "likes_used": len(likes) if 'likes' in locals() else 0,
                "dislikes_used": len(dislikes) if 'dislikes' in locals() else 0,
                "persona": persona, "keywords_positive": pos, "keywords_negative": neg,
                "like_images": likes if 'likes' in locals() else [],
                "dislike_images": dislikes if 'dislikes' in locals() else [],
                "note": f"hard failure: {type(e).__name__}"
            })

    # save
    Path(OUT_JSONL).parent.mkdir(parents=True, exist_ok=True)
    with open(OUT_JSONL, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    flat = pd.DataFrame([{
        "user_id": r["user_id"],
        "likes_used": r["likes_used"],
        "dislikes_used": r["dislikes_used"],
        "persona": r["persona"],
        "keywords_positive": ", ".join(r["keywords_positive"]),
        "keywords_negative": ", ".join(r["keywords_negative"]),
        "note": r.get("note","")
    } for r in results])
    flat.to_csv(OUT_CSV, index=False)

    bad = sum(1 for r in results if r.get("note"))
    print(f"\nWrote {len(results)} users.")
    print(f"  JSONL: {OUT_JSONL}")
    print(f"  CSV:   {OUT_CSV}")
    print(f"  Fallbacks used for {bad} users (kept outputs valid).")




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
main()

Obtaining user profiles: 100%|██████████| 210/210 [24:24<00:00,  6.97s/it]


Wrote 210 users.
  JSONL: ./personas_llava_collegetry.jsonl
  CSV:   ./personas_llava_collegetry.csv
  Fallbacks used for 24 users (kept outputs valid).


## Partition and store the data in convenient structure

In [4]:

def store_images_by_user_like(data, savedir, imgdir, RANDOM_SEED=42):
    for worker_id in data.worker_id.unique():
        path = os.path.join(savedir, str(worker_id))
        likes_path = os.path.join(path, "like")
        dislikes_path = os.path.join(path, "dislike")

        os.makedirs(likes_path, exist_ok=True)
        os.makedirs(dislikes_path, exist_ok=True)

        # Get rows for this worker
        worker_rows = data[data["worker_id"] == worker_id]

        # Sample up to 20 liked and 20 disliked images
        liked = worker_rows[worker_rows["rating"] == True].sample(
            n=min(20, (worker_rows["rating"] == True).sum()),
            random_state=RANDOM_SEED
        )
        disliked = worker_rows[worker_rows["rating"] == False].sample(
            n=min(20, (worker_rows["rating"] == False).sum()),
            random_state=RANDOM_SEED
        )

        # Copy sampled liked images
        for _, row in liked.iterrows():
            img_path = os.path.join(imgdir, row["imagePair"])
            if os.path.exists(img_path):
                shutil.copy(img_path, likes_path)
            else:
                print(f"Warning: Image not found at {img_path}")

        # Copy sampled disliked images
        for _, row in disliked.iterrows():
            img_path = os.path.join(imgdir, row["imagePair"])
            if os.path.exists(img_path):
                shutil.copy(img_path, dislikes_path)
            else:
                print(f"Warning: Image not found at {img_path}")


In [5]:
data = pd.read_csv("../data/flickr/processed/train/train_usrthrs_0.csv")
data["rating"] = data["score"] > 3
RANDOM_SEED = 42

savedir = "../data/flickr/processed/vlm-profiling/train"
imgdir = "../data/flickr/raw/FLICKR-AES-001/40K/"

store_images_by_user_like(data, savedir, imgdir, RANDOM_SEED=RANDOM_SEED)

In [8]:
test_data = pd.read_csv("../data/flickr/processed/test/test_usrthrs_0.csv")
test_data["rating"] = test_data["score"] > 3
RANDOM_SEED = 43

test_savedir = "../data/flickr/processed/vlm-profiling/test"
test_imgdir = "../data/flickr/raw/FLICKR-AES-001/40K/"

store_images_by_user_like(test_data, test_savedir, test_imgdir, RANDOM_SEED=RANDOM_SEED)

## Produce profiles

### Produce captions for all images

In [40]:
image_captions

{'image': ['farm1_176_396297967_94dae8ad97.jpg'],
 'caption': ['Asian man wearing glasses and a blue shirt']}

In [29]:
test_image_path = "../data/flickr/raw/FLICKR-AES-001/40K/farm2_1416_1247921410_0be0f4f6df.jpg"
img = Image.open(test_image_path).convert("RGB")
desc = tag_image_llava(img)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


USER:  
You are a tagging engine.
Return ONLY a JSON object and nothing else. No prose, no code fences.
The JSON must have exactly these fields:
{"caption": str, "objects":[str], "attributes":[str], "styles":[str], "colors":[str], "scenes":[str]
Rules: lowercase; <=10 items TOTAL across all lists; strings only; no nulls. ASSISTANT: {"caption": "Gold statue of a man sitting on a bench", "objects": ["statue", "bench"], "attributes": ["gold", "sitting"], "styles": ["decorative"], "colors": ["gold"], "scenes": ["outdoor"]}


## Generate by Prompt

In [4]:
from diffusers import StableDiffusionPipeline
import torch
import pandas as pd

In [2]:
model_id = "runwayml/stable-diffusion-v1-5"                                                                                                                                                                                                                
pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")
pipe.safety_checker = None
generator = torch.Generator("cuda").manual_seed(0)

C:\Users\Gabriel\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
topic_caption_data = pd.read_csv("../data/flickr/processed/content_captions_gpt4omini.csv")
eval_data = pd.read_csv("../data/flickr/processed/test_usrthrs_100.csv")

In [6]:
images = []
prompts = []
paths = []
for _, row in eval_data.iterrows():
    image_pair = row["imagePair"]
    path = "../data/flickr/raw/40K/" + image_pair
    prompt  = topic_caption_data[topic_caption_data["Image"] == image_pair]["Content_Caption"].iloc[0]
    image = pipe(
    prompt=prompt,
    num_inference_steps=100,
    generator=generator,
    ).images
    images.append(image)
    prompts.append(prompts)
    paths.append(path)

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\diffusers\models\attention_processor.py:1279: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  hidden_states = F.scaled_dot_product_attention(


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
import os
save_dir = "../data/flickr/evaluation/baselines/topic_recommender"
os.makedirs(save_dir, exist_ok=True)

for i, img in enumerate(images):
    image_pair = eval_data.iloc[i]["imagePair"]
    filepath = os.path.join(save_dir, image_pair)
    img[0].save(filepath)

## Generate by LLM-Profiler

In [2]:
import os
import json
from diffusers import StableDiffusionPipeline
import torch

user_profiles_dir = "../data/flickr/evaluation/baselines/llm_profiling/user_description"
user_profiles = {}

In [3]:


# Loop through all files in the directory
for filename in os.listdir(user_profiles_dir):
    if filename.startswith("user_") and filename.endswith(".json"):
        user_id = int(filename.split("_")[1].split(".")[0])
        filepath = os.path.join(user_profiles_dir, filename)
        
        with open(filepath, "r") as f:
            data = json.load(f)
            description = data.get("description", "")
            keywords = data.get("keywords", "")
            user_profiles[user_id] = {
                "description": description,
                "keywords": keywords
            }

In [4]:
model_id = "runwayml/stable-diffusion-v1-5"                                                                                                                                                                                                                
pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")
pipe.safety_checker = None
generator = torch.Generator("cuda").manual_seed(0)

C:\Users\Gabriel\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
user_profiles

{0: {'description': "The user's visual preferences lean towards natural beauty and serene landscapes. They seem to appreciate vibrant colors, especially in flowers, and the intricate details of nature. Many liked images feature elements of tranquility such as lakes, mountains, and greenery, indicating a preference for the outdoors and peaceful environments. They also seem to enjoy architectural elements, particularly those with historical significance. Conversely, the disliked images tend to focus more on crowded social scenes, urban settings, and subjects that appear chaotic or cluttered. Overall, the user appears to favor simplicity and elegance in design, favoring nature and artistry over busyness and crowds.",
  'keywords': 'nature, serenity, landscapes, colors, flowers, tranquility, architecture, simplicity, elegance, outdoors'},
 1: {'description': "The user demonstrates a strong preference for nature and organic subjects, favoring images that evoke tranquility and beauty, such a

In [5]:
import torch
from tqdm import tqdm

# Settings
images_per_batch = 10
total_images_per_user = 50
num_batches = total_images_per_user // images_per_batch
seed = 42

# Store all images per user
for user_id, profile in tqdm(user_profiles.items()):
    all_images = []
    prompt = "A photo the user likes. Here are some of their keywords: " + profile["keywords"]
    prompts = [prompt] * images_per_batch  # prompt batch

    generator = torch.Generator(device="cuda").manual_seed(seed + user_id)

    for batch_idx in range(num_batches):
        result = pipe(
            prompt=prompts,
            num_inference_steps=50,
            generator=generator
        )
        all_images.extend(result.images)

    profile["generated_images"] = all_images
    profile["prompts_used"] = [prompt] * total_images_per_user


  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\diffusers\models\attention_processor.py:1279: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  hidden_states = F.scaled_dot_product_attention(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 94/94 [7:29:06<00:00, 286.67s/it]


In [8]:
user_profiles

{0: {'description': "The user's visual preferences lean towards natural beauty and serene landscapes. They seem to appreciate vibrant colors, especially in flowers, and the intricate details of nature. Many liked images feature elements of tranquility such as lakes, mountains, and greenery, indicating a preference for the outdoors and peaceful environments. They also seem to enjoy architectural elements, particularly those with historical significance. Conversely, the disliked images tend to focus more on crowded social scenes, urban settings, and subjects that appear chaotic or cluttered. Overall, the user appears to favor simplicity and elegance in design, favoring nature and artistry over busyness and crowds.",
  'keywords': 'nature, serenity, landscapes, colors, flowers, tranquility, architecture, simplicity, elegance, outdoors',
  'generated_images': [<PIL.Image.Image image mode=RGB size=512x512>,
   <PIL.Image.Image image mode=RGB size=512x512>,
   <PIL.Image.Image image mode=RGB

In [13]:
savedir = "../data/flickr/evaluation/baselines/llm_profiling/generated_images"
for user, user_dict in user_profiles.items():
    user_images = user_dict["generated_images"]
    user_dir = os.path.join(savedir, str(user))
    os.makedirs(user_dir, exist_ok=True)
    for i, image in enumerate(user_images):
        image_name = f"img_{i}.png"
        filepath = os.path.join(user_dir, image_name)
        image.save(filepath)

## Generate Front Cover pics

In [28]:
from diffusers import StableDiffusionPipeline
import torch
import pandas as pd

model_id = "runwayml/stable-diffusion-v1-5"                                                                                                                                                                                                                
pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")
pipe.safety_checker = None
generator = torch.Generator("cuda").manual_seed(0)



selected_idx_gt = [
'farm9_8643_15892994369_136b246684_b.jpg', 
'farm1_666_21609487146_df2568268f_b.jpg',  
    'farm5_4105_4969491488_b980275a88_b.jpg',   
    'farm9_8446_7982543560_586fe4ea05_b.jpg',   
    'farm1_168_475755642_811ee945c8.jpg', 
    'farm7_6068_6074760400_d7fbb9f023.jpg',  
]

images = []
prompts = []
paths = []
for impath in selected_idx_gt:
    path = "../data/flickr/raw/40K/" + impath
    prompt  = topic_caption_data[topic_caption_data["Image"] == impath]["Content_Caption"].iloc[0]
    image = pipe(
    prompt=prompt,
    num_inference_steps=100,
    generator=generator,
    ).images
    images.append(image[0])
    prompts.append(prompt)
    paths.append(path)

C:\Users\Gabriel\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
prompts

['A close-up of a bullfinch perched on a branch, showcasing its distinctive orange and black plumage. The bird is surrounded by blurred green foliage, with soft branches in the background.',
 'A serene lake scene featuring various boats docked at a marina. The water is calm, reflecting the blue sky and fluffy white clouds above. Lush green trees line the shore, creating a tranquil and picturesque setting.',
 "A close-up view of a pair of modern running shoes resting on a slatted surface, with a person's leg partially visible beside them. The shoes feature a combination of gray and yellow colors, showcasing a sleek design.",
 'A close-up portrait of an elderly man with a weathered face, deep-set eyes, and a prominent white mustache and beard. He is wearing a dark hat and has a serious expression, suggesting wisdom and experience.',
 'A close-up view of lilac flowers in varying stages of bloom, featuring clusters of delicate purple buds and fully opened petals. The background is softly b

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)
topic_caption_data = pd.read_csv("../data/flickr/processed/content_captions_gpt4omini.csv")
user_profile_data = pd.read_csv("./all_94_user_new_descriptions_77.csv")
selected_idx_gt = {
    'a bullfinch perched on a branch':1, 
    'serene lake scene':1,  
    'a pair of modern running shoes':20,   
    'an elderly man with a weathered face':20,   
    'flowers in varying stages of bloom':51, 
    'car displayed under a tent at a vintage car show':51,  
}


images = []
prompts = []
paths = []
for cprompt, user in selected_idx_gt.items():

    prompt  = [cprompt + " " + user_profile_data[user_profile_data["user_id"]==1]["description"].iloc[0]]
    image = pipe(
    prompt=prompt,
    num_inference_steps=100,
    generator=generator,
    ).images
    images.append(image[0])
    prompts.append(prompt)
    paths.append(path)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [50]:
import os
save_dir = "../data/flickr/processed/llm-profiling/front_cover"
os.makedirs(save_dir, exist_ok=True)

for i, img in enumerate(images):
    image_name = f"img_{i}.png"
    filepath = os.path.join(save_dir, image_name)
    img.save(filepath)